<a href="https://colab.research.google.com/github/hollipista/MetaCortex-AAEE/blob/main/02b_MetaCortex_AAEE_anal_portf_ipynb_m%C3%A1solata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import statsmodels.api as sm
import gc
import requests

# Database handler
#from sqlalchemy import create_engine, text, MetaData, Table, Column, Integer, Date, String, FLOAT

# Necessary for compatibility
#from sqlalchemy.dialects.postgresql.base import PGDialect
#PGDialect._get_server_version_info = lambda *args: (9, 2)

# Necessary for standard t-stat transformation
from statistics import NormalDist

In [2]:
# előkészített adatok letöltése és kicsomagolása
!wget -O CRSP_data_ready.csv.zip https://www.dropbox.com/scl/fi/nut6nhrndvy5k2cnjt1pb/CRSP_data_ready.csv.zip?rlkey=1zvidp8l8s60pyqjlskh0o8pk&dl=0
!wget -O sp500_data_ready.csv.zip https://www.dropbox.com/scl/fi/di2e7ug7c96wekqg450vw/sp500_data_ready.csv.zip?rlkey=42byzuo3p1m4rhqsxrzsx5kwf&dl=0
!wget -O ff3_data_ready.csv.zip https://www.dropbox.com/scl/fi/nc9weel4i2hhfpa2tf1qp/ff3_data_ready.csv.zip?rlkey=rmz7g2522c5o0tx0aegdfrewf&dl=0

!unzip -n /content/CRSP_data_ready.csv.zip
!unzip -n /content/sp500_data_ready.csv.zip
!unzip -n /content/ff3_data_ready.csv.zip

--2023-11-25 15:17:12--  https://www.dropbox.com/scl/fi/nut6nhrndvy5k2cnjt1pb/CRSP_data_ready.csv.zip?rlkey=1zvidp8l8s60pyqjlskh0o8pk
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9ba87f882c05677eb315086f32.dl.dropboxusercontent.com/cd/0/inline/CIM46qMEHXWMLCiUdtIFvCxP3st9DPanjojt2PHw37efcDSwbQdedr9mREkjGQ3emn5sllzNu7acuQb17NvF3Ukz5rsY4jO_E97T1f6dz0Th8LrjI4HHxgJdbfRDHN8XsIAoOH8A7URZei-kS6ikjoF6/file# [following]
--2023-11-25 15:17:13--  https://uc9ba87f882c05677eb315086f32.dl.dropboxusercontent.com/cd/0/inline/CIM46qMEHXWMLCiUdtIFvCxP3st9DPanjojt2PHw37efcDSwbQdedr9mREkjGQ3emn5sllzNu7acuQb17NvF3Ukz5rsY4jO_E97T1f6dz0Th8LrjI4HHxgJdbfRDHN8XsIAoOH8A7URZei-kS6ikjoF6/file
Resolving uc9ba87f882c05677eb315086f32.dl.dropboxusercontent.com (uc9ba87f882c05677eb315086f32.dl.dropboxusercontent.com)... 162.125.5

In [3]:
url = 'https://raw.githubusercontent.com/saintsjd/sic4-list/master/sic-codes.csv'

# HTTP GET kérés küldése a fájl letöltéséhez
response = requests.get(url)

if response.status_code == 200:
    # A fájl tartalmának kiírása, ha a letöltés sikeres volt
    with open('sic-codes.csv', 'wb') as file:
        file.write(response.content)
    print('A fájl sikeresen letöltve.')
else:
    print('Hiba történt a letöltés során. Hibaüzenet:', response.status_code)

A fájl sikeresen letöltve.


In [4]:
url = 'https://www2.census.gov/programs-surveys/cbp/technical-documentation/records-layouts/sic-code-descriptions/sic86_87.txt'

# HTTP GET kérés küldése a fájl letöltéséhez
response = requests.get(url)

if response.status_code == 200:
    # A fájl tartalmának kiírása, ha a letöltés sikeres volt
    with open('sic-codes2.csv', 'wb') as file:
        file.write(response.content)
    print('A fájl sikeresen letöltve.')
else:
    print('Hiba történt a letöltés során. Hibaüzenet:', response.status_code)



A fájl sikeresen letöltve.


In [5]:
# Dataframe-ek betöltése
crsp = pd.read_csv("/content/CRSP_data_ready.csv", sep=",")
crsp = crsp[crsp.columns[1:]]
gc.collect() # memória takarítás
sp500 = pd.read_csv("/content/sp500_data_ready.csv", sep=",")
ff3 = pd.read_csv("/content/ff3_data_ready.csv", sep=",")
#sic = pd.read_csv("/content/sic-codes.csv", sep=',')

#sic = pd.read_csv("/content/sic-codes2.csv", sep='\t')
#sic["SIC"] = pd.to_numeric(sic['SIC'], errors='coerce')
#sic = sic.rename(columns={'NAME   *indicates codes used only in 1986': 'Description'})
#sic = sic[~sic['SIC'].isna()]
#sic['SIC'] = sic['SIC'].astype(int)
#sic.dtypes

###############################################################################
# Oké, én megint órákat szívtam érdemi eredmény nélkül, illet az érdemi eredmény az,
# hogy két forrást is találtam a SIC kódokra és persze egyik lista sem teljes, pedig
# mindkettő hivatalos forrásból van.
# Elegánsabb lenne mergelni és utána szűrni, de ezt elengedem most. Alább bemásol-
# tam azokat a SIC 4-digit kódokat, amikre futtatni szeretnénk, majd egyesével
# futtatunk rájuk. Bocs.

<ipython-input-5-3b72b31a479f>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp = pd.read_csv("/content/CRSP_data_ready.csv", sep=",")


In [6]:
crsp.head()

,date,PERMNO,CUSIP,SIC,SH,EX,Prc,Tvol,Ret,Shr,AbsPrc,Size,lnRet
0,2000-01-03,10001,36720410,4920,11,3.0,8.5625,1721.0,0.007353,2450.0,8.5625,9.951236,0.007326
1,2000-01-04,10001,36720410,4920,11,3.0,8.4375,1080.0,-0.014599,2450.0,8.4375,9.936529,-0.014707
2,2000-01-05,10001,36720410,4920,11,3.0,8.5625,1711.0,0.014815,2450.0,8.5625,9.951236,0.014706
3,2000-01-06,10001,36720410,4920,11,3.0,8.5000,580.0,-0.007299,2450.0,8.5000,9.943909,-0.007326
4,2000-01-07,10001,36720410,4920,11,3.0,8.4375,1406.0,-0.007353,2450.0,8.4375,9.936529,-0.007380


In [7]:
crsp=crsp[["date", "PERMNO", "SIC", "Size", "lnRet"]]
gc.collect()

0

3482	Small Arms Ammunition
3483	Ammunition, Except for Small Arms
3484	Small Arms

7381	Detective, Guard, and Armored Car Services
7382	Security Systems Services

6311	Life Insurance
6321	Accident and Health Insurance
6324	Hospital and Medical Service Plans
6331	Fire, Marine, and Casualty Insurance
6351	Surety Insurance
6361	Title Insurance
6371	Pension, Health, and Welfare Funds
6399	Insurance Carriers, Not Elsewhere Classified

6531	Real Estate Agents and Managers
6798	Real Estate Investment Trusts

4512 Air Transportation, Scheduled
4513 Air Courier Services
4522 Air Transportation, Nonscheduled
4581 Airports, Flying Fields, and Airport Terminal Services

4724 Travel Agencies
4725 Tour Operators

# Légi közlekedés

In [11]:
sic_values = [[3482, 3483, 3484],
              [7381, 7382],
              [6311, 6321, 6324, 6331, 6351, 6361, 6371, 6399],
              [6531, 6798],
              [4512, 4513, 4522, 4581],
              [4724, 4725]]
sic_labels = ["Arms, ammo", "Security, guard", "Insurance",
              "Real est. agent and op.", "Air transport", "Travel"]

yr = 2011

i = 0

for i, sic_list in enumerate(sic_values):

  # Iparági szűrés
  vwp = crsp.loc[crsp["SIC"].isin(sic_list)].copy()

  # S&P500 merge-ölés
  vwp = vwp.merge(
          sp500.rename(columns={"return_index": "market"})[["date", "market"]],
          how="inner",
          left_on="date",
          right_on="date"
      )

  # FF3 faktorok merge-ölés
  vwp = vwp.merge(
          ff3[["date", "Mkt-RF", "SMB", "HML", "RF"]],
          how="inner",
          left_on="date",
          right_on="date"
      )

  # Rendezés részvényenként, dátumra
  vwp=vwp.sort_values(by=['PERMNO', 'date'])

  # évre szűrés
  yrstart = str(yr) + "-01-01"
  yrend = str(yr) + "-12-31"
  vwp = vwp.loc[(vwp["date"] >= yrstart) & (vwp["date"] <= yrend)].copy()
  vwp["date"] = pd.to_datetime(vwp["date"]).dt.date

  # Value weighted portfolio képzése
  vwp['weight'] = vwp['Size']/vwp.groupby('date')['Size'].transform('sum')
  vwp['portflnret'] = vwp['lnRet']*vwp['weight']

  vwp['portflnret'] = vwp.groupby('date')['portflnret'].transform('sum')
  vwp.drop(columns=['weight', 'lnRet'], inplace=True)
  vwp.sort_values('date', ascending=True, inplace=True)
  vwp = vwp.drop_duplicates(subset='date', keep='first')
  vwp['PERMNO'] = 'Portfolio'

  # becslési és eseményablak számítása,
  # most itt megelőző naptól t+5-ig
  # becslési pedig 150 munkanappal korábbtól 22 munkanappal korábbig
  event_date = pd.to_datetime(str(yr) + "-09-11").date()
  if event_date.isoweekday() in [6, 7]:
          event_date = (event_date + pd.offsets.BusinessDay()).date()
  event_date
  event_dates = [event_date + pd.offsets.BusinessDay(x) for x in range(-1, 6, 1)]
  estimation_end = (event_date - pd.offsets.BusinessDay(22)).date()
  estimation_start = (event_date - pd.offsets.BusinessDay(150)).date()

  # OLS Market modellre és FF3 modellre, majd becslés, AR és t-stat számítás,
  # és mindezt beírva táblába
  vwp["estimation"] = np.where((vwp["date"] >= estimation_start) & (vwp["date"] <= estimation_end), 1, 0)
  vwp["event"] = np.where(vwp["date"].isin([x.date() for x in event_dates]), 1, 0)
  vwp = vwp[(vwp["estimation"] == 1) | (vwp["event"] == 1)]
  estimation = vwp["estimation"] == 1
  # Market modell
  model = sm.OLS(endog=vwp[estimation]["portflnret"], exog=sm.tools.add_constant(vwp[estimation]["market"]))
  results = model.fit()
  vwp["estimated"] = vwp.apply(lambda x: results.params.const + x["market"] * results.params.market, axis=1)
  vwp["ar"] = vwp["portflnret"] - vwp["estimated"]
  vwp["portflnret"] - vwp["estimated"]
  # FF3 modell
  vwp = vwp.rename(columns={'Mkt-RF': 'MktRF'})
  modelff3 = sm.OLS(endog=vwp[estimation]["portflnret"], exog=sm.tools.add_constant(vwp[estimation][["MktRF", "SMB", "HML"]]))
  resultsff3 = modelff3.fit()
  vwp["estimatedff3"] = vwp.apply(lambda x: results.params.const + x["MktRF"] * resultsff3.params.MktRF +
                                  x['SMB'] * resultsff3.params.SMB + x['HML'] * resultsff3.params.HML, axis=1)
  vwp["arff3"] = vwp["portflnret"] - vwp["estimatedff3"]
  # T-1 és T0 esetén a becslési ablak alapján számolom a szigmát
  sigma1nap = np.sqrt((1/vwp[vwp['estimation'] == 1].shape[0] * ((vwp[vwp['estimation'] == 1]['ar'] ** 2).sum())))
  sigma1napff3 = np.sqrt((1/vwp[vwp['estimation'] == 1].shape[0] * ((vwp[vwp['estimation'] == 1]['arff3'] ** 2).sum())))
  # itt leszűrűnk az eseményablakra
  vwp = vwp.loc[vwp["event"] == 1]
  # majd T1..T5 kummulált AR-t számolunk, Tetlock eljárásához hasonlóan
  # (vagyis ha lenne T-1 vagy T0 AR, akkor T+5-re korrigálna)
  vwp["window"] = (pd.Series(range(vwp.shape[0])) - 1).values
  sigmawin = np.sqrt((5+1)*(1/(5-1)*(vwp[vwp['window'] > 0]['ar'] ** 2).sum()))
  sigmawinff3 = np.sqrt((5+1)*(1/(5-1)*(vwp[vwp['window'] > 0]['arff3'] ** 2).sum()))
  # kummulált hozam és tesztstatisztikák számítása (modellenként)
  vwp["car"] = np.where(vwp['window'] == 5, (vwp[vwp['window'] > 0]['ar']).sum(), np.nan)
  vwp['TS'] = np.where(vwp['window'] < 1, vwp['ar'] / sigma1nap, np.nan)
  vwp['TS'] = np.where(vwp['window'] == 5, (vwp[vwp['window'] > 0]['ar']).sum() / sigmawin, vwp['TS'])
  vwp["carff3"] = np.where(vwp['window'] == 5, (vwp[vwp['window'] > 0]['arff3']).sum(), np.nan)
  vwp['TSff3'] = np.where(vwp['window'] < 1, vwp['arff3'] / sigma1napff3, np.nan)
  vwp['TSff3'] = np.where(vwp['window'] == 5, (vwp[vwp['window'] > 0]['arff3']).sum() / sigmawinff3, vwp['TSff3'])
  vwp.to_csv('0' + str(i) + '_' + str(yr) + '_' + sic_labels[i] + '.csv', index=False, sep=";")
  i = i + 1

<ipython-input-11-e6db5bf632ce>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vwp["window"] = (pd.Series(range(vwp.shape[0])) - 1).values
<ipython-input-11-e6db5bf632ce>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vwp["car"] = np.where(vwp['window'] == 5, (vwp[vwp['window'] > 0]['ar']).sum(), np.nan)
<ipython-input-11-e6db5bf632ce>:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [ ]:
vwp